In [108]:
import pandas as pd
import os

# # Show current working directory - the directory where all your files are saved by default
# os.getcwd()

# Set path for new working directory
path = "C:/Users/Sarah/Faris Stuff/USM Data Science Masters Files/CDS503/Week 5 - 16 Mar/Data"
os.chdir(path) 

# # Check to see if current directory has changed
# os.getcwd()

# Read data from CSV to a data frame named df
df = pd.read_csv('bankdata.csv') 
# Display the data
df

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
0,ID12101,48,FEMALE,INNER_CITY,17546.00,NO,1,NO,NO,NO,NO,YES
1,ID12102,40,MALE,TOWN,30085.10,YES,3,YES,NO,YES,YES,NO
2,ID12103,51,FEMALE,INNER_CITY,16575.40,YES,0,YES,YES,YES,NO,NO
3,ID12104,23,FEMALE,TOWN,20375.40,YES,3,NO,NO,YES,NO,NO
4,ID12105,57,FEMALE,RURAL,50576.30,YES,0,NO,YES,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
595,ID12696,61,FEMALE,INNER_CITY,47025.00,NO,2,YES,YES,YES,YES,NO
596,ID12697,30,FEMALE,INNER_CITY,9672.25,YES,0,YES,YES,YES,NO,NO
597,ID12698,31,FEMALE,TOWN,15976.30,YES,0,YES,YES,NO,NO,YES
598,ID12699,29,MALE,INNER_CITY,14711.80,YES,0,NO,YES,NO,YES,NO


In [109]:
# remove id column as it's useless to tell if the customer buys a PEP or not
# Setting axis=1 means we want to drop a column, if axis=0, it's to remove a row)
df = df.drop('id', axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          600 non-null    int64  
 1   sex          600 non-null    object 
 2   region       600 non-null    object 
 3   income       600 non-null    float64
 4   married      600 non-null    object 
 5   children     600 non-null    int64  
 6   car          600 non-null    object 
 7   save_act     600 non-null    object 
 8   current_act  600 non-null    object 
 9   mortgage     600 non-null    object 
 10  pep          600 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 51.7+ KB


In [110]:
# Gotta convert sex from string (male, female) to numerical representation (0, 1s) as some algos require numerical input data
# to do that, use something called Label Encoder

# Import LabelEncoder
from sklearn import preprocessing

# Create LabelEncoder
le = preprocessing.LabelEncoder()

# Convert string categories into numbers for sex
df['sex'] = le.fit_transform(df['sex'])
df['region'] = le.fit_transform(df['region'])
df['married'] = le.fit_transform(df['married'])
df['car'] = le.fit_transform(df['car'])
df['save_act'] = le.fit_transform(df['save_act'])
df['current_act'] = le.fit_transform(df['current_act'])
df['mortgage'] = le.fit_transform(df['mortgage'])
df['pep'] = le.fit_transform(df['pep'])
df.head()

,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
0,48,0,0,17546.0,0,1,0,0,0,0,1
1,40,1,3,30085.1,1,3,1,0,1,1,0
2,51,0,0,16575.4,1,0,1,1,1,0,0
3,23,0,3,20375.4,1,3,0,0,1,0,0
4,57,0,1,50576.3,1,0,0,1,0,0,0


In [111]:
# Initialize arrays to keep results of the testing to get best results later
test = []
result = []

### Baseline

In [112]:
# Indicate the target column
target = df['pep']

# features column are basically everything else that's not the target
features = df.drop('pep', axis = 1)

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split the dataset into training and test set, test size is 20%, training size is 80%
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

#### Naive Bayes

In [113]:
# Import function for k-fold cross validation
from sklearn.model_selection import cross_val_score
# Import the Gaussian, Bernoulli, and Multinomial Naive Bayes classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Create a Gaussian, Bernoulli, and Multinomial Naive Bayes classifier with default parameters
gnb = GaussianNB()
bnb = BernoulliNB(binarize = 0.0)
mnb = MultinomialNB()

# Use 10-fold cross validation to perform training and validation on the training set
# Parameter scoring = 'accuracy' will compute accuracy
gnb_scores = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
bnb_scores = cross_val_score(bnb, x_train, y_train, cv = 5, scoring = 'accuracy')
mnb_scores = cross_val_score(mnb, x_train, y_train, cv = 5, scoring = 'accuracy')

# Show train scores, fit, test, print results
print('BASELINE')
print('Gaussian Naive Bayes')
print('Accuracy (Train): ', gnb_scores.mean())
test.append('Baseline GNB Train Accuracy')
result.append(gnb_scores.mean())

gnb.fit(x_train, y_train)
gnb_test_predict = gnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, gnb_test_predict))
test.append('Baseline GNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, gnb_test_predict))
print()

print('Bernoulli Naive Bayes')
print('Accuracy (Train): ', bnb_scores.mean())
test.append('Baseline BNB Train Accuracy')
result.append(bnb_scores.mean())

bnb.fit(x_train, y_train)
bnb_test_predict = bnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, bnb_test_predict))
test.append('Baseline BNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, bnb_test_predict))
print()

print('Multinomial Naive Bayes')
print('Accuracy (Train): ', mnb_scores.mean())
test.append('Baseline MNB Train Accuracy')
result.append(mnb_scores.mean())

mnb.fit(x_train, y_train)
mnb_test_predict = mnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, mnb_test_predict))
test.append('Baseline MNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, mnb_test_predict))

BASELINE
Gaussian Naive Bayes
Accuracy (Train):  0.6395833333333334
Accuracy (Test):  0.5916666666666667

Bernoulli Naive Bayes
Accuracy (Train):  0.5479166666666667
Accuracy (Test):  0.525

Multinomial Naive Bayes
Accuracy (Train):  0.5979166666666667
Accuracy (Test):  0.5416666666666666


### Data Discretization Test

In [114]:
df.describe()

,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,42.395000,0.500000,1.231667,27524.031217,0.660000,1.011667,0.493333,0.690000,0.758333,0.348333,0.456667
std,14.424947,0.500417,1.286113,12899.468246,0.474104,1.056752,0.500373,0.462879,0.428451,0.476840,0.498534
min,18.000000,0.000000,0.000000,5014.210000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,0.000000,17264.500000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,42.000000,0.500000,1.000000,24925.300000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,55.250000,1.000000,3.000000,36172.675000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,67.000000,1.000000,3.000000,63130.100000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Cut and QCut Methods

In [115]:
# Create another data frame and to make sure df_new does not modify df
df_cut = df.copy()
df_qcut = df.copy()

# Divide the data into 4 bins
# Divide the data into 4 bins with equal range automatically
df_cut['age_bin'] = pd.cut(df_cut['age'], bins = 4)
df_cut['income_bin'] = pd.cut(df_cut['income'], bins = 4)

df_qcut['age_bin'] = pd.qcut(df_qcut['age'], q = 4, precision = 0)
df_qcut['income_bin'] = pd.qcut(df_qcut['age'], q = 4, precision = 0)

# View the count of each bin
print(df_cut['age_bin'].value_counts())
print(df_cut['income_bin'].value_counts())

print(df_qcut['age_bin'].value_counts())
print(df_qcut['income_bin'].value_counts())

# label encode the new age_bin and income_bin columns
df_cut['age_bin'] = le.fit_transform(df_cut['age_bin'])
df_cut['income_bin'] = le.fit_transform(df_cut['income_bin'])

df_qcut['age_bin'] = le.fit_transform(df_qcut['age_bin'])
df_qcut['income_bin'] = le.fit_transform(df_qcut['income_bin'])

# Remove the original age and income columns to not double consider these columns in the features
df_cut = df_cut.drop(['age', 'income'], axis = 1)
df_cut

age_bin
(17.951, 30.25]    156
(54.75, 67.0]      153
(30.25, 42.5]      147
(42.5, 54.75]      144
Name: count, dtype: int64
income_bin
(19543.182, 34072.155]    240
(4956.094, 19543.182]     192
(34072.155, 48601.127]    112
(48601.127, 63130.1]       56
Name: count, dtype: int64
age_bin
(17.0, 30.0]    156
(55.0, 67.0]    150
(30.0, 42.0]    147
(42.0, 55.0]    147
Name: count, dtype: int64
income_bin
(17.0, 30.0]    156
(55.0, 67.0]    150
(30.0, 42.0]    147
(42.0, 55.0]    147
Name: count, dtype: int64


,sex,region,married,children,car,save_act,current_act,mortgage,pep,age_bin,income_bin
0,0,0,0,1,0,0,0,0,1,2,0
1,1,3,1,3,1,0,1,1,0,1,1
2,0,0,1,0,1,1,1,0,0,2,0
3,0,3,1,3,0,0,1,0,0,0,1
4,0,1,1,0,0,1,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,2,1,1,1,1,0,3,2
596,0,0,1,0,1,1,1,0,0,0,0
597,0,3,1,0,1,1,0,0,1,1,0
598,1,0,1,0,0,1,0,1,0,0,0


##### DF_Cut

In [116]:
# Indicate the target column
target_cut = df_cut['pep']

# features column are basically everything else that's not the target
features = df_cut.drop('pep', axis = 1)

# Split the dataset into training and test set, test size is 20%, training size is 80%
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

In [117]:
# Import function for k-fold cross validation
from sklearn.model_selection import cross_val_score
# Import the Gaussian, Bernoulli, and Multinomial Naive Bayes classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Create a Gaussian, Bernoulli, and Multinomial Naive Bayes classifier with default parameters
gnb = GaussianNB()
bnb = BernoulliNB(binarize = 0.0)
mnb = MultinomialNB()

# Use 10-fold cross validation to perform training and validation on the training set
# Parameter scoring = 'accuracy' will compute accuracy
gnb_scores = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
bnb_scores = cross_val_score(bnb, x_train, y_train, cv = 5, scoring = 'accuracy')
mnb_scores = cross_val_score(mnb, x_train, y_train, cv = 5, scoring = 'accuracy')

# Show train scores, fit, test, print results
print('CUT')
print('Gaussian Naive Bayes')
print('Accuracy (Train): ', gnb_scores.mean())
test.append('CUT GNB Train Accuracy')
result.append(gnb_scores.mean())

gnb.fit(x_train, y_train)
gnb_test_predict = gnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, gnb_test_predict))
test.append('CUT GNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, gnb_test_predict))
print()

print('Bernoulli Naive Bayes')
print('Accuracy (Train): ', bnb_scores.mean())
test.append('CUT BNB Train Accuracy')
result.append(bnb_scores.mean())

bnb.fit(x_train, y_train)
bnb_test_predict = bnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, bnb_test_predict))
test.append('CUT BNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, bnb_test_predict))
print()

print('Multinomial Naive Bayes')
print('Accuracy (Train): ', mnb_scores.mean())
test.append('CUT MNB Train Accuracy')
result.append(mnb_scores.mean())

mnb.fit(x_train, y_train)
mnb_test_predict = mnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, mnb_test_predict))
test.append('CUT MNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, mnb_test_predict))

CUT
Gaussian Naive Bayes
Accuracy (Train):  0.6333333333333333
Accuracy (Test):  0.625

Bernoulli Naive Bayes
Accuracy (Train):  0.625
Accuracy (Test):  0.6166666666666667

Multinomial Naive Bayes
Accuracy (Train):  0.60625
Accuracy (Test):  0.5333333333333333


##### DF_QCut

In [118]:
# Indicate the target column
target_cut = df_qcut['pep']

# features column are basically everything else that's not the target
features = df_qcut.drop('pep', axis = 1)

# Split the dataset into training and test set, test size is 20%, training size is 80%
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

In [119]:
# Import function for k-fold cross validation
from sklearn.model_selection import cross_val_score
# Import the Gaussian, Bernoulli, and Multinomial Naive Bayes classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Create a Gaussian, Bernoulli, and Multinomial Naive Bayes classifier with default parameters
gnb = GaussianNB()
bnb = BernoulliNB(binarize = 0.0)
mnb = MultinomialNB()

# Use 10-fold cross validation to perform training and validation on the training set
# Parameter scoring = 'accuracy' will compute accuracy
gnb_scores = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
bnb_scores = cross_val_score(bnb, x_train, y_train, cv = 5, scoring = 'accuracy')
mnb_scores = cross_val_score(mnb, x_train, y_train, cv = 5, scoring = 'accuracy')

# Show train scores, fit, test, print results
print('QCUT')
print('Gaussian Naive Bayes')
print('Accuracy (Train): ', gnb_scores.mean())
test.append('QCUT GNB Train Accuracy')
result.append(gnb_scores.mean())

gnb.fit(x_train, y_train)
gnb_test_predict = gnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, gnb_test_predict))
test.append('QCUT GNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, gnb_test_predict))
print()

print('Bernoulli Naive Bayes')
print('Accuracy (Train): ', bnb_scores.mean())
test.append('QCUT BNB Train Accuracy')
result.append(bnb_scores.mean())

bnb.fit(x_train, y_train)
bnb_test_predict = bnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, bnb_test_predict))
test.append('QCUT BNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, bnb_test_predict))
print()

print('Multinomial Naive Bayes')
print('Accuracy (Train): ', mnb_scores.mean())
test.append('QCUT MNB Train Accuracy')
result.append(mnb_scores.mean())

mnb.fit(x_train, y_train)
mnb_test_predict = mnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, mnb_test_predict))
test.append('QCUT MNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, mnb_test_predict))

QCUT
Gaussian Naive Bayes
Accuracy (Train):  0.6145833333333334
Accuracy (Test):  0.5666666666666667

Bernoulli Naive Bayes
Accuracy (Train):  0.6333333333333333
Accuracy (Test):  0.6416666666666667

Multinomial Naive Bayes
Accuracy (Train):  0.5958333333333333
Accuracy (Test):  0.55


#### One Hot Encoding

In [120]:
# Columns to one-hot encode
categorical_cols = ['sex', 'region', 'married', 'car', 'save_act', 'current_act', 'mortgage', 'age_bin', 'income_bin']

# Use get_dummies() function to perform one hot encoding on sex column
df_enc_cut = pd.get_dummies(df_cut, columns = categorical_cols)
df_enc_qcut = pd.get_dummies(df_qcut, columns = categorical_cols)

##### DF_Cut

In [121]:
# Indicate the target column
target_cut = df_enc_cut['pep']

# features column are basically everything else that's not the target
features = df_enc_cut.drop('pep', axis = 1)

# Split the dataset into training and test set, test size is 20%, training size is 80%
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

In [122]:
# Import function for k-fold cross validation
from sklearn.model_selection import cross_val_score
# Import the Gaussian, Bernoulli, and Multinomial Naive Bayes classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Create a Gaussian, Bernoulli, and Multinomial Naive Bayes classifier with default parameters
gnb = GaussianNB()
bnb = BernoulliNB(binarize = 0.0)
mnb = MultinomialNB()

# Use 10-fold cross validation to perform training and validation on the training set
# Parameter scoring = 'accuracy' will compute accuracy
gnb_scores = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
bnb_scores = cross_val_score(bnb, x_train, y_train, cv = 5, scoring = 'accuracy')
mnb_scores = cross_val_score(mnb, x_train, y_train, cv = 5, scoring = 'accuracy')

# Show train scores, fit, test, print results
print('One Hot Encode CUT')
print('Gaussian Naive Bayes')
print('Accuracy (Train): ', gnb_scores.mean())
test.append('OHE CUT GNB Train Accuracy')
result.append(gnb_scores.mean())

gnb.fit(x_train, y_train)
gnb_test_predict = gnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, gnb_test_predict))
test.append('OHE CUT GNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, gnb_test_predict))
print()

print('Bernoulli Naive Bayes')
print('Accuracy (Train): ', bnb_scores.mean())
test.append('OHE CUT BNB Train Accuracy')
result.append(bnb_scores.mean())

bnb.fit(x_train, y_train)
bnb_test_predict = bnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, bnb_test_predict))
test.append('OHE CUT BNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, bnb_test_predict))
print()

print('Multinomial Naive Bayes')
print('Accuracy (Train): ', mnb_scores.mean())
test.append('OHE CUT MNB Train Accuracy')
result.append(mnb_scores.mean())

mnb.fit(x_train, y_train)
mnb_test_predict = mnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, mnb_test_predict))
test.append('OHE CUT MNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, mnb_test_predict))

One Hot Encode CUT
Gaussian Naive Bayes
Accuracy (Train):  0.6270833333333332
Accuracy (Test):  0.6166666666666667

Bernoulli Naive Bayes
Accuracy (Train):  0.6
Accuracy (Test):  0.6

Multinomial Naive Bayes
Accuracy (Train):  0.6208333333333333
Accuracy (Test):  0.5833333333333334


##### DF_QCut

In [123]:
# Indicate the target column
target_cut = df_enc_qcut['pep']

# features column are basically everything else that's not the target
features = df_enc_qcut.drop('pep', axis = 1)

# Split the dataset into training and test set, test size is 20%, training size is 80%
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

In [124]:
# Import function for k-fold cross validation
from sklearn.model_selection import cross_val_score
# Import the Gaussian, Bernoulli, and Multinomial Naive Bayes classifiers
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Create a Gaussian, Bernoulli, and Multinomial Naive Bayes classifier with default parameters
gnb = GaussianNB()
bnb = BernoulliNB(binarize = 0.0)
mnb = MultinomialNB()

# Use 10-fold cross validation to perform training and validation on the training set
# Parameter scoring = 'accuracy' will compute accuracy
gnb_scores = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
bnb_scores = cross_val_score(bnb, x_train, y_train, cv = 5, scoring = 'accuracy')
mnb_scores = cross_val_score(mnb, x_train, y_train, cv = 5, scoring = 'accuracy')

# Show train scores, fit, test, print results
print('One Hot Encode QCUT')
print('Gaussian Naive Bayes')
print('Accuracy (Train): ', gnb_scores.mean())
test.append('OHE QCUT GNB Train Accuracy')
result.append(gnb_scores.mean())

gnb.fit(x_train, y_train)
gnb_test_predict = gnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, gnb_test_predict))
test.append('OHE QCUT GNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, gnb_test_predict))
print()

print('Bernoulli Naive Bayes')
print('Accuracy (Train): ', bnb_scores.mean())
test.append('OHE QCUT BNB Train Accuracy')
result.append(bnb_scores.mean())

bnb.fit(x_train, y_train)
bnb_test_predict = bnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, bnb_test_predict))
test.append('OHE QCUT BNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, bnb_test_predict))
print()

print('Multinomial Naive Bayes')
print('Accuracy (Train): ', mnb_scores.mean())
test.append('OHE QCUT MNB Train Accuracy')
result.append(mnb_scores.mean())

mnb.fit(x_train, y_train)
mnb_test_predict = mnb.predict(x_test)
print("Accuracy (Test): ", metrics.accuracy_score(y_test, mnb_test_predict))
test.append('OHE QCUT MNB Test Accuracy')
result.append(metrics.accuracy_score(y_test, mnb_test_predict))

One Hot Encode QCUT
Gaussian Naive Bayes
Accuracy (Train):  0.6291666666666667
Accuracy (Test):  0.5833333333333334

Bernoulli Naive Bayes
Accuracy (Train):  0.6249999999999999
Accuracy (Test):  0.5666666666666667

Multinomial Naive Bayes
Accuracy (Train):  0.6083333333333333
Accuracy (Test):  0.5666666666666667


In [129]:
best_result = 0
best_resultloc = 0

for i in result:
    # print(i)
    if i > best_result:
        best_result = i

best_resultloc = result.index(best_result)
best_test = test[best_resultloc]

# print()
print(f'Best Result: {best_result}')
print(f'Best Result location in array: {best_resultloc}')
print(f'Best Test: {best_test}')

Best Result: 0.6416666666666667
Best Result location in array: 15
Best Test: QCUT BNB Test Accuracy
